In [69]:
import numpy as np
import torch
from torch.utils import data
from torch import nn
import torch.optim as optim
from tqdm import tqdm
import argparse


In [2]:
class TimeSeriesChunkDataset(data.Dataset):
    def __init__(self, x, y, context):
        super(TimeSeriesChunkDataset, self).__init__()
        self.x = x
        self.y = y
        self.context = context
        self.points_per_series = self.x.shape[1] - self.context + 1
        
    def __len__(self):
        return self.x.shape[0] * self.points_per_series

    def __getitem__(self, index):
        index_series = index // self.points_per_series
        index_point = index % self.points_per_series
        return_x = self.x[index_series, index_point:index_point+self.context, :]
        return_y = np.argmax(self.y[index_series, :])
        return return_x, return_y

In [3]:
class Classifier(nn.Module):
    def __init__(self, layers_size, dim_out, dropout=0.3):
        super(Classifier, self).__init__()
        self.layers = []
        self.activs = []
        self.dropouts = []
        self.layers_size = layers_size
        
        for i in range(len(layers_size)-1):
            self.layers.append(nn.Linear(layers_size[i], layers_size[i+1]))
            self.activs.append(nn.ReLU())
            if i < len(layers_size)-2:
                self.dropouts.append(nn.Dropout(p=dropout))

        self.layers.append(nn.Linear(layers_size[-1], dim_out))
        self.nlayer = len(self.layers)
        self.nactivs = len(self.activs)
        self.ndropouts = len(self.dropouts)

        self.layers = nn.ModuleList(self.layers)
        
    def forward(self, x):
        out = x
        for i in range(self.nlayer):
            out = self.layers[i](out)
            if i < self.nactivs:
                out = self.activs[i](out)
            if i < self.ndropouts:
                out = self.dropouts[i](out)

        return out


In [4]:
class Discriminator(nn.Module):
    '''
    credit: from https://github.com/martinmamql/complex_da
    '''
    def __init__(self, feature_dim, d_out):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(

            nn.Linear(feature_dim, feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),
        ) 
        self.fc = nn.Linear(3200, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # x: [bs, seq, feature_dim]
        x = self.net(x)
        bs = x.shape[0]
        x = x.reshape(bs, -1)
        out = self.sigmoid(self.fc(x))
        return out

In [102]:
class Generator(nn.Module):
    def __init__(self, **kwargs):
        super(Generator, self).__init__()
        self.transformer_layer_args = {'d_model':2, 'nhead':1, 'dim_feedforward':1024, 'dropout':0.1, 'activation':'gelu'}
        self.transformer_args = {'num_layers':3, 'norm':None}
        self.transformer_layer_args.update(kwargs['transformer_layer'])
        self.transformer_args.update(kwargs['transformer'])
        
        self.transformer_layer = nn.TransformerEncoderLayer(**self.transformer_layer_args)
        self.transformer = nn.TransformerEncoder(self.transformer_layer, **self.transformer_args)
        
    def forward(self, x):
        out = self.transformer(x)
        return out
        

In [139]:
class SourceDomainClassifier(nn.Module):
    def __init__(self, **kwargs):
        super(SourceDomainClassifier, self).__init__()
        self.net = nn.Sequential(
            Generator(**(kwargs["generator"])),
            nn.Flatten(),
            Classifier(**(kwargs['classifier']))
        )
        
    def forward(self, x):
        x = self.net(x)
        return x
        

In [148]:
def get_accuracy(preds, label):
    class_preds = torch.argmax(preds, dim=1)
    correct = torch.eq(class_preds, label).float()
    acc = correct.sum()
    return acc

def inference(model, dataloader, device, testing=False):
    model.eval()

    if testing: 
        result = []
    else:
        inference_acc = 0.0

    num_data = 0.0
    with torch.no_grad():
        for key, (x_batch, y_batch) in enumerate(dataloader):
            num_data += y_batch.shape[0]
            x_batch = x_batch.to(device)
            y_batch = y_batch.long().to(device)
            preds = model(x_batch)
            if testing:
                result.extend(torch.argmax(nn.functional.softmax(preds, dim=1), dim=1).cpu().numpy())
            else:
                inference_acc += get_accuracy(preds, y_batch.squeeze_()).item()
    

    if testing:
        return result
    else:
        return inference_acc / num_data
    
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.00)
    

In [143]:
def train(model, train_dataloader, vali_dataloader, lr, n_epochs, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
#     scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.7, patience=3)
    loss_fn = nn.CrossEntropyLoss()
    train_loss_ = []
    train_acc_ = []
    vali_acc_ = []
    
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        train_acc = 0.0

        num_data = 0.0
        num_batches = len(train_dataloader)
        for batches, (x_batch, y_batch) in tqdm(enumerate(train_dataloader), total=num_batches):
            model.train()
            num_data += y_batch.shape[0]
            x_batch = x_batch.to(device)
            y_batch = y_batch.long().to(device)
            optimizer.zero_grad()
            preds = model(x_batch)
            loss = loss_fn(preds, y_batch.squeeze_())
            acc = get_accuracy(preds, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_acc += acc.item()
            
            if batches > 1 and batches % int(num_batches/3) == 0:
                vali_acc = inference(model, vali_dataloader, device)
                print("validation_acc: ", vali_acc)

        vali_acc = inference(model, vali_dataloader, device)
        train_loss_.append(train_loss/num_data)
        train_acc_.append(train_acc/num_data)
        vali_acc_.append(vali_acc)
        # scheduler.step(vali_acc)
        name = "model_" + str(epoch) + ".t7"   
        np.save("train_loss_.npy",train_loss_)
        np.save("train_acc_.npy",train_acc_)
        np.save("vali_acc_.npy",vali_acc_)
        torch.save(model.state_dict(), name)

        print("epoch {}: train_loss: {}, train_acc: {}, vali_acc: {}".format(epoch, train_loss/num_data, train_acc/num_data, vali_acc))


In [146]:
#local only

class fake_args():
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

        
args = fake_args(data_path='./data_unzip/', 
                 task='3A', 
                 batch_size=30,
                 epochs=2,
                 lr=1e-3,
                 context=10)

args.task = "processed_file_3Av2.pkl" if args.task == "3A" else "processed_file_3E.pkl"
args.data_path = args.data_path + args.task

In [149]:
if __name__ == "__main__":
    cuda = torch.cuda.is_available()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    data_dict = np.load(args.data_path, allow_pickle=True)
    
    # TODO: to be commented
    data_dict['tr_data'] = data_dict['tr_data'][:10]
    data_dict['tr_data'] = data_dict['tr_data'][:10]
    
    # split train data and validation data
    np.random.seed(seed=0)
    indices = np.random.permutation(data_dict['tr_data'].shape[0])
    train_x = data_dict['tr_data'][indices[:int(indices.shape[0]*0.9)],:,:].astype("float32")
    train_y = data_dict['tr_lbl'][indices[:int(indices.shape[0]*0.9)],:].astype("float32")
    vali_x = data_dict['tr_data'][indices[int(indices.shape[0]*0.9):],:,:].astype("float32")
    vali_y = data_dict['tr_lbl'][indices[int(indices.shape[0]*0.9):],:].astype("float32")

    
    train_dataset = TimeSeriesChunkDataset(train_x, train_y, args.context)
    train_dataloader = data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, pin_memory=True, num_workers=4)
    vali_dataset = TimeSeriesChunkDataset(vali_x, vali_y, args.context)
    vali_dataloader = data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, pin_memory=True, num_workers=4)
    
    model_args = {
        'classifier': {
            'layers_size': [args.context*2, 100, 100], 
            'dim_out': 50 if args.task == '3A' else 65
        },
        'generator':{
            'transformer_layer': {},
            'transformer': {}
        }
    }
    
    
    
    source_classifier = SourceDomainClassifier(**model_args)
    source_classifier.apply(init_weights)
    
    train(source_classifier, train_dataloader, vali_dataloader, args.lr, args.epochs, device)
    



 35%|███▍      | 167/478 [00:05<00:26, 11.67it/s]

validation_acc:  0.3329841469376353



 68%|██████▊   | 327/478 [00:10<00:15,  9.91it/s]

validation_acc:  0.3674139255534604



100%|██████████| 478/478 [00:15<00:00, 30.03it/s]

...


 68%|██████▊   | 324/478 [00:10<00:15,  9.85it/s]

validation_acc:  0.3907395767860884



100%|██████████| 478/478 [00:16<00:00, 29.23it/s]

validation_acc:  0.3951393253718835


epoch 2: train_loss: 0.05015768307332129, train_acc: 0.3864795027585725, vali_acc: 0.3920664850897409


In [122]:
for name, param in source_classifier.named_parameters():
    if param.requires_grad:
        print (name, param.data, param.dtype)

transformation.transformer_layer.self_attn.in_proj_weight tensor([[-0.0365, -0.8347],
        [-0.1136, -0.7268],
        [-0.8044, -0.5697],
        [ 0.5512, -0.5657],
        [ 0.5618, -0.2861],
        [-0.3158,  0.6102]]) torch.float32
transformation.transformer_layer.self_attn.in_proj_bias tensor([0., 0., 0., 0., 0., 0.]) torch.float32
transformation.transformer_layer.self_attn.out_proj.weight tensor([[-1.0541, -0.2033],
        [-0.3923, -1.0161]]) torch.float32
transformation.transformer_layer.self_attn.out_proj.bias tensor([0., 0.]) torch.float32
transformation.transformer_layer.linear1.weight tensor([[ 0.0526,  0.0169],
        [-0.0167, -0.0424],
        [ 0.0643, -0.0719],
        ...,
        [ 0.0023, -0.0527],
        [-0.0460, -0.0076],
        [-0.0416, -0.0403]]) torch.float32
transformation.transformer_layer.linear1.bias tensor([0., 0., 0.,  ..., 0., 0., 0.]) torch.float32
transformation.transformer_layer.linear2.weight tensor([[-0.0639,  0.0704, -0.0497,  ..., -0.02